In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np



In [2]:
eval_interval = 500
learning_rate = 2e-4
batch_size = 64
block_size = 256
max_iters = 6000
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 8
n_layer = 6
dropout = 0.1

with open('adventures_1MB.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
le = LabelEncoder()
le.fit(list(set(text)))
vocab_size = len(le.classes_)

encoded_data = le.transform(list(text))
data = torch.tensor(encoded_data, dtype=torch.long)

train_data_np, val_data_np = train_test_split(data.numpy(), test_size=0.1, shuffle=False)
train_data = torch.tensor(train_data_np, dtype=torch.long)
val_data = torch.tensor(val_data_np, dtype=torch.long)

def get_batch(split):
    split_data = train_data if split == 'train' else val_data
    ix = torch.randint(len(split_data) - block_size, (batch_size,))
    x = torch.stack([split_data[i:i+block_size] for i in ix])
    y = torch.stack([split_data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel().to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.ones((1, 1), dtype=torch.long, device=device)
output = model.generate(context, max_new_tokens=2000)[0].tolist()
print(le.inverse_transform(output))

4.84617 M parameters
step 0: train loss 4.6316, val loss 4.6286
step 500: train loss 2.3781, val loss 2.4656
step 1000: train loss 1.9228, val loss 2.0388
step 1500: train loss 1.6776, val loss 1.8050
step 2000: train loss 1.5160, val loss 1.6538
step 2500: train loss 1.4089, val loss 1.5772
step 3000: train loss 1.3342, val loss 1.5163
step 3500: train loss 1.2729, val loss 1.4769
step 4000: train loss 1.2305, val loss 1.4472
step 4500: train loss 1.1938, val loss 1.4331
step 5000: train loss 1.1660, val loss 1.4160
step 5500: train loss 1.1351, val loss 1.4017
step 5999: train loss 1.1127, val loss 1.3962
['\n' 'a' 'n' ... 'u' 't' ' ']


In [4]:

# prompt = "The "

# input_ids = le.transform(list(prompt))
# input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)


# generated = model.generate(input_tensor, max_new_tokens=100)[0].tolist()

# output_text = ''.join(le.inverse_transform(generated))

# print(output_text)

In [6]:

prompt = "The"

input_ids = le.transform(list(prompt))
input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)


generated = model.generate(input_tensor, max_new_tokens=1500)[0].tolist()

output_text = ''.join(le.inverse_transform(generated))

print(output_text)

/tmp/ipython-input-2225826372.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)


Thence,
in the same abjourney according intested, my inconrimation with In a
rein agest any Bacgan indulgence his abitation and heaven! Duly
laugh without situation reat a grow mother astonishment.

The captain too work, atmentipies the presumption of a partner, who
acquainted me with the trust trick and offerrience, to waise sent to
that were styleman. “Randol do you gaftin!” connoly prejudiced the goose
upon me now; thip you gostly scarces, “Are not a mallices, ladieus
young in the gentleman, before Strap’s summonically, know, d—you
supppos with sendedom of adventure to paid her, I must in on this markablim
eyes too man a few hotmer and barbarkfames.” I; asgemed the looks of releat.
As I have in the foot of light, but she lipped his wildness of
ay, and smile I became so minto eagh Thouffle by the Wiffe?” continued
and Lord Mons. That explained with keachban each characted a same begon
us betrange, by the mistaken is a disappointment of which we in escampes
to each) on mentioned on th